# Genome dataset

The tag genome is a data structure that contains tag relevance scores for movies. The structure is a dense matrix where each movie in the genome has a value for every tag in the genome. We decided to add these two datasets at a later stage because the results of the model were not very satisfactory.

### Imports

In [22]:
import os

import pandas as pd

from src.utils.const import DATA_DIR

### Useful path to data

In [23]:
ROOT_DIR = os.path.join(os.getcwd(), '..')
RAW_DIR = os.path.join(ROOT_DIR, DATA_DIR, 'raw')

## Data Acquisition

We are assumed that the notebooks are explored in order, so these two datasets should already be stored inside the raw folder.

## Data Pre-processing

If we want to use these datasets, we have to create a new `DataFrame()` that for each sample contains the values of all the __tags__ with respect to the __movieId__. For this reason we have first to read the _genome-scores_ and _genome-tags_.

### genome-scores.csv

In [24]:
# Read
genome_scores = pd.read_csv(
    os.path.join(RAW_DIR, 'genome-scores.csv'),
    encoding='utf-8',
    dtype={'movieId':'int32', 'tagId':'int32', 'relevance':'float32'}
)

genome_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14862528 entries, 0 to 14862527
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int32  
 1   tagId      int32  
 2   relevance  float32
dtypes: float32(1), int32(2)
memory usage: 170.1 MB


### genome-tags.csv

In [25]:
# Read
genome_tags = pd.read_csv(
    os.path.join(RAW_DIR, 'genome-tags.csv'),
    encoding='utf-8',
    dtype={'tagId':'int32', 'tag':'string'}
)

genome_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tagId   1128 non-null   int32 
 1   tag     1128 non-null   string
dtypes: int32(1), string(1)
memory usage: 13.3 KB


### Merge

The next step calculates the left union between the two datasets.

In [26]:
tags_relevance = genome_scores.merge(genome_tags, on='tagId', how='left')

In [27]:
tags_relevance.head()

,movieId,tagId,relevance,tag
0,1,1,0.02900,007
1,1,2,0.02375,007 (series)
2,1,3,0.05425,18th century
3,1,4,0.06875,1920s
4,1,5,0.16000,1930s


Thanks to the `pivot()` function we can obtain exactly what we wanted. Now, we have for each __movieId__, all the __tags__ with their relevance.

In [28]:
tags_relevance=(tags_relevance
                .pivot(index='movieId', columns='tag', values='relevance')
                .reset_index()
                .astype({'movieId': 'int32'}))

In [29]:
tags_relevance.head()

tag,movieId,007,007 (series),18th century,1920s,1930s,1950s,1960s,1970s,1980s,...,world politics,world war i,world war ii,writer's life,writers,writing,wuxia,wwii,zombie,zombies
0,1,0.02900,0.02375,0.05425,0.06875,0.16000,0.19525,0.07600,0.25200,0.22750,...,0.03775,0.02250,0.04075,0.03175,0.12950,0.04550,0.02000,0.03850,0.09125,0.02225
1,2,0.03625,0.03625,0.08275,0.08175,0.10200,0.06900,0.05775,0.10100,0.08225,...,0.04775,0.02050,0.01650,0.02450,0.13050,0.02700,0.01825,0.01225,0.09925,0.01850
2,3,0.04150,0.04950,0.03000,0.09525,0.04525,0.05925,0.04000,0.14150,0.04075,...,0.05800,0.02375,0.03550,0.02125,0.12775,0.03250,0.01625,0.02125,0.09525,0.01750
3,4,0.03350,0.03675,0.04275,0.02625,0.05250,0.03025,0.02425,0.07475,0.03750,...,0.04900,0.03275,0.02125,0.03675,0.15925,0.05225,0.01500,0.01600,0.09175,0.01500
4,5,0.04050,0.05175,0.03600,0.04625,0.05500,0.08000,0.02150,0.07375,0.02825,...,0.05375,0.02625,0.02050,0.02125,0.17725,0.02050,0.01500,0.01550,0.08875,0.01575


In [30]:
tags_relevance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13176 entries, 0 to 13175
Columns: 1129 entries, movieId to zombies
dtypes: float32(1128), int32(1)
memory usage: 56.7 MB


In [31]:
print(f'Merged genomes dimensionality: {tags_relevance.shape}')

Merged genomes dimensionality: (13176, 1129)


At the end, we can see that the cardinality of this dataset is smaller than the movies' dataset. We will drop all the samples that are not linked with `tags_relevance` in the next notebook.